In [1]:
import pickle
import glob
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import log_loss
import xgboost as xgb

# load feats
train_x,test_x = [],[]
for feat in sorted(glob.glob('../features/*.pkl')):
#     if 'tfidf' in feat or 'lr' in feat or 'mnb' in feat:
#         continue
    print('file path',feat)
    a,b = pickle.load(open(feat,'rb'))
    print(a.shape,b.shape)
    train_x.append(a)
    test_x.append(b)
train_x = np.nan_to_num(np.hstack(train_x))
test_x = np.nan_to_num(np.hstack(test_x))
print(train_x.shape)
    
# load y
train = pd.read_csv("../input/train.csv")
list_classes = ["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]
train_y = train[list_classes].values.astype('int')
print(train_x.shape)

file path ../features/cnn2d_muse_1_feat.pkl
(159571, 6) (153164, 6)
file path ../features/cnn_glove_1_feat.pkl
(159571, 6) (153164, 6)
file path ../features/cnn_gru_glove_1_feat.pkl
(159571, 6) (153164, 6)
file path ../features/cnn_muse_adj_1_feat_de_fr.pkl
(159571, 6) (153164, 6)
file path ../features/cudnn_gru_glove_1_feat.pkl
(159571, 6) (153164, 6)
file path ../features/lr_feat1.pkl
(159571, 6) (153164, 6)
file path ../features/lr_feat2.pkl
(159571, 6) (153164, 6)
file path ../features/lstm_glove_1_feat.pkl
(159571, 6) (153164, 6)
file path ../features/mnb_feat1.pkl
(159571, 6) (153164, 6)
file path ../features/mnb_feat2.pkl
(159571, 6) (153164, 6)
file path ../features/other_feat.pkl
(159571, 16) (153164, 16)
file path ../features/tfidf_feat1.pkl
(159571, 30) (153164, 30)
file path ../features/tfidf_feat2.pkl
(159571, 30) (153164, 30)
(159571, 136)
(159571, 136)


In [2]:
from sklearn.model_selection import KFold
def simple_ens(model_name,k=3,rnd=233):
    kf = KFold(n_splits=k, shuffle=True, random_state=rnd)
    test_pred = np.zeros((153164,6))
    cache_test_pred = np.zeros((153164,6))
    single_best = 100
    single_best_pred = None
    all_train_loss_l,all_val_loss_l = 0,0
    
    for train_index, test_index in kf.split(train_x):
        # x,y
        curr_x,curr_y = train_x[train_index],train_y[train_index]
        hold_out_x,hold_out_y = train_x[test_index],train_y[test_index]
        val_loss_l,train_loss_l = 0,0
        d_test = xgb.DMatrix(test_x)
        
        # share params
        params = {
                'colsample_bytree': 0.9,
                'subsample': 0.9,
                'eta': 0.05,
                'max_depth': 3,
                'eval_metric':'logloss',
                'objective':'binary:logistic',
                'scale_pos_weight':0.9,
                'colsample_bylevel':0.9,
                'colsample_bytree':0.9,
            
                }
        
        # train for each class
        for i in range(6):
            d_train = xgb.DMatrix(curr_x, curr_y[:,i])
            d_valid = xgb.DMatrix(hold_out_x, hold_out_y[:,i])
            watchlist = [(d_train, 'train'), (d_valid, 'valid')]

            model = xgb.train(params, d_train, 1000, watchlist,
                              early_stopping_rounds=50,
                              verbose_eval=2000)
            print(i)
            try:
                curr_train_loss = log_loss(curr_y[:,i],model.predict(d_train))
                curr_val_loss = log_loss(hold_out_y[:,i],model.predict(d_valid))
                print(curr_train_loss,curr_val_loss)
                val_loss_l += curr_val_loss
                train_loss_l += curr_train_loss
            except:
                pass
            curr_test_pred = model.predict(d_test)
            
            test_pred[:,i] += curr_test_pred
            cache_test_pred[:,i] += curr_test_pred
            
        # avg 6 class
        train_loss_l = train_loss_l/6
        val_loss_l = val_loss_l/6
        print('this fold avg train',train_loss_l,'avg val',val_loss_l)
        
        # save best one fold result
        if val_loss_l < single_best:
            single_best = val_loss_l
            single_best_pred = cache_test_pred
            print('new single best')
        
        cache_test_pred = np.zeros((153164,6))
        
        # avg k fold
        all_train_loss_l += train_loss_l/k
        all_val_loss_l += val_loss_l/k
        print('========================')
    test_pred = test_pred/k
    print('all train avg',all_train_loss_l,'all val avg',all_val_loss_l)
    return test_pred, single_best_pred

print('done')

done


In [3]:
xgb_res,b = simple_ens('xgb',k=3)
sample_submission = pd.read_csv("../input/sample_submission.csv")
sample_submission[list_classes] = xgb_res
sample_submission.to_csv("../results/xgb_ens_new_csv_fold3.gz", index=False, compression='gzip')
print(sample_submission.head())
print('save done')

# sample_submission = pd.read_csv("../input/sample_submission.csv")
# sample_submission[list_classes] = b
# sample_submission.to_csv("../results/xgb_ens_new_csv_fold5_single_best.gz", index=False, compression='gzip')
# print(sample_submission.head())
# print('save done')


[0]	train-logloss:0.649274	valid-logloss:0.649234
Multiple eval metrics have been passed: 'valid-logloss' will be used for early stopping.

Will train until valid-logloss hasn't improved in 50 rounds.
Stopping. Best iteration:
[501]	train-logloss:0.071191	valid-logloss:0.082106

0
0.0701568420884 0.0821179643652
[0]	train-logloss:0.645651	valid-logloss:0.64565
Multiple eval metrics have been passed: 'valid-logloss' will be used for early stopping.

Will train until valid-logloss hasn't improved in 50 rounds.
Stopping. Best iteration:
[252]	train-logloss:0.017053	valid-logloss:0.019857

1
0.0162010616285 0.0199005792786
[0]	train-logloss:0.64691	valid-logloss:0.646863
Multiple eval metrics have been passed: 'valid-logloss' will be used for early stopping.

Will train until valid-logloss hasn't improved in 50 rounds.
Stopping. Best iteration:
[197]	train-logloss:0.03974	valid-logloss:0.042024

2
0.0386101292062 0.0420608456124
[0]	train-logloss:0.644744	valid-logloss:0.64479
Multiple eva

In [4]:
xgb_res,b = simple_ens('xgb',k=5)
sample_submission = pd.read_csv("../input/sample_submission.csv")
sample_submission[list_classes] = xgb_res
sample_submission.to_csv("../results/xgb_ens_new_csv_fold5.gz", index=False, compression='gzip')
print(sample_submission.head())
print('save done')


[0]	train-logloss:0.649265	valid-logloss:0.649249
Multiple eval metrics have been passed: 'valid-logloss' will be used for early stopping.

Will train until valid-logloss hasn't improved in 50 rounds.
Stopping. Best iteration:
[665]	train-logloss:0.069653	valid-logloss:0.081304

0
0.0688445198045 0.0813453399248
[0]	train-logloss:0.645637	valid-logloss:0.645638
Multiple eval metrics have been passed: 'valid-logloss' will be used for early stopping.

Will train until valid-logloss hasn't improved in 50 rounds.
Stopping. Best iteration:
[313]	train-logloss:0.016406	valid-logloss:0.020273

1
0.0157270076659 0.0203551785381
[0]	train-logloss:0.646913	valid-logloss:0.646849
Multiple eval metrics have been passed: 'valid-logloss' will be used for early stopping.

Will train until valid-logloss hasn't improved in 50 rounds.
Stopping. Best iteration:
[187]	train-logloss:0.040232	valid-logloss:0.041595

2
0.039210018001 0.0416413461433
[0]	train-logloss:0.644761	valid-logloss:0.644829
Multiple 

[0]	train-logloss:0.645623	valid-logloss:0.645671
Multiple eval metrics have been passed: 'valid-logloss' will be used for early stopping.

Will train until valid-logloss hasn't improved in 50 rounds.
Stopping. Best iteration:
[183]	train-logloss:0.018168	valid-logloss:0.021837

1
0.0173433447913 0.0218899111135
[0]	train-logloss:0.646857	valid-logloss:0.646975
Multiple eval metrics have been passed: 'valid-logloss' will be used for early stopping.

Will train until valid-logloss hasn't improved in 50 rounds.
Stopping. Best iteration:
[281]	train-logloss:0.037586	valid-logloss:0.044077

2
0.0366404006411 0.0441094233609
[0]	train-logloss:0.644742	valid-logloss:0.644793
Multiple eval metrics have been passed: 'valid-logloss' will be used for early stopping.

Will train until valid-logloss hasn't improved in 50 rounds.
Stopping. Best iteration:
[215]	train-logloss:0.00464	valid-logloss:0.008533

3
0.004068917139 0.00858213218386
[0]	train-logloss:0.647823	valid-logloss:0.647918
Multiple 